In [ ]:
# Install packages below needed for the lab
!pip install snowflake-snowpark-python pandas notebook scikit-learn cachetools pyarrow==10.0.1
!pip install azure-ai-ml
!pip install mltable

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()
# # Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="<azure subscription number>",
    resource_group_name="<resource group name>",
    workspace_name="<azureml workspace name>",
)

In [ ]:
import mltable
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get the latest version of the data asset
# Note: The version was set in the previous code cell.
data_asset = ml_client.data.get(name="orderhistory", version="1")
# create a table
tbl = mltable.load(f"azureml:/{data_asset.id}")
df = tbl.to_pandas_dataframe()
df['MED_AGE'] = df['MED_AGE'].apply(str)
df

In [ ]:
import pandas as pd
import requests
import json

# Azure OpenAI API endpoint URL
endpoint_url = "<flow endpoint>"

# API key or authentication token for accessing the endpoint
api_key = "<flow endpoint key>"

# Headers for the HTTP request
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response_data = []

# Iterate through rows and make requests
for index, row in df.iterrows():
    input_purchases = row["RECENT_PURCHASES"]
    input_age = row["MED_AGE"]

    # Construct payload using the input text
    payload = {
        "order_history": input_purchases,
        "med_age": input_age,
        "max_tokens": 100
    }

    # Convert payload to JSON and encode as bytes
    payload_bytes = json.dumps(payload).encode("utf-8")

    # Send the POST request
    response = requests.post(endpoint_url, data=payload_bytes, headers=headers)

    # Process the response
    if response.status_code == 200:
        result = response.json()
        generated_text = result.get("output", "No generated text found")
        response_data.append({"Input": input_purchases, "Generated": generated_text})
    else:
        print(f"Request for '{input_purchases}' failed with status code:", response.status_code)

# Create a DataFrame from the response data list
response_df = pd.DataFrame(response_data)
response_df['ID'] = df['ID']

# Print the resulting DataFrame
response_df

In [ ]:
from snowflake.snowpark import Session
connection_parameters = {
    "account": "<snowflake account>", # e.g. birmlgk-ld03678 (orgname-accountIdentifier)
    "user": "<username>", 
    "password": "<password>",
    "role": "ACCOUNTADMIN",
    "database": "edopoku_db",
    "schema": "PUBLIC",
    "warehouse": "edopokuwh"
    }
session = Session.builder.configs(connection_parameters).create()
session.write_pandas(response_df, table_name='Marketing_PROMO', auto_create_table=True, overwrite=True)